# Data preprocessing of updated Udacity dataset

In [ ]:
# At a first look, the dataset contains 22241 images from which we have the follwing classes 
# with their corresponding number of instances (appearances within images):
# car - 127900
# truck - 7194
# pedestrian - 21491
# trafficLight-Red - 13673
# trafficLight-RedLeft - 3482
# trafficLight-Yellow - 541
# trafficLight-YellowLeft - 28
# trafficLight-Green - 10838
# trafficLight - 5101
# biker - 3704
# For creating a balanced dataset we dropped the labels for biker and truck as they were very little compared to the
# other classes and merge the classes containg trafficLights
# We obtained the following classes with their corresponding instances:
# car - 127900
# pedestrian - 21491
# trafficLight - 34277
# Note: around 3000 images from the total were found to be background images (with no classes in th)

In [ ]:
os.chdir('C:\\Users\\andre\\Desktop\\Object detection project\\TFODCourse')
os.chdir('Tensorflow\\workspace\\export')

In [ ]:
# Converting the already created .json file back into a dictionary in order to see which images contain pedestrians/trafficLights and cars
# and delete the rest of the images, for balancing the labels
# importing the module
import json
 
# Opening JSON file
with open('_dictionary.json') as json_file:
    data = json.load(json_file)
 
    # Print the type of data variable
    print("Type:", type(data))

In [ ]:
# Iterating through the dictionary and creating another one where only the the labels for traffic light appear
trafficLights_filename = []
count_trafficLights = 0
for filename in data.keys():
    for label in data.get(filename):
        if label == "trafficLight":
            count_trafficLights += 1
            trafficLights_filename.append(filename)
# trafficLight could have appeared multiple times as a value in the same key, so we have to make a list with unique filenames
trafficLights_filename = list(set(trafficLights_filename))
# 5101 labels for trafficLight in 3175 images

In [ ]:
# Iterating through the dictionary and creating another one where only the the labels for pedestrian appear
pedestrian_filename = []
count_pedestrians = 0
for filename in data.keys():
    for label in data.get(filename):
        if label == "pedestrian":
            count_pedestrians += 1
            pedestrian_filename.append(filename)
# pedestrian could have appeared multiple times as a value in the same key, so we have to make a list with unique filenames
pedestrian_filename = list(set(pedestrian_filename))
# 21491 labels for trafficLight in 7030 images

In [ ]:
# Count how many times the label for cars appear where trafficLight appears
count_cars_light = 0
for filename in trafficLights_filename:
    for label in data.get(filename):
        if label == "car":
            count_cars_light += 1
# 16743 cars and 5101 trafficLights in 3175 images

In [ ]:
# Count how many times the label for cars appear where pedestrian appears
count_cars_pedestrians = 0
for filename in pedestrian_filename:
    for label in data.get(filename):
        if label == "car":
            count_cars_pedestrians += 1
# 34934 cars and 21491 pedestrians in 7030 images
# 56425 rows + 1

# Wrong conclusion. We need to take another look on the trafficLight labels 

In [ ]:
# Conclusion:
# we will use the second part of the dataset for cars and pedestrian
# pedestrian_filename

In [ ]:
# Moving the training images with cars and pedestrian to another folder
for file in pedestrian_filename:
    shutil.move(file, 'train')

In [ ]:
# Creating a separate .csv file for the new training images
old_csv = pd.read_csv('_annotations.csv')
# print(type(new_csv))

In [ ]:
# Opening an empty .csv and writing only the images that are needed
new_csv = pd.read_csv('out.csv')
# df1 = df1.append(new_csv.iloc[2], ignore_index = True)
for index, filename in enumerate(old_csv["filename"]):
    for name in pedestrian_filename:
        if filename == name:
            new_csv = new_csv.append(old_csv.iloc[index], ignore_index = True)
new_csv.to_csv('new_csv.csv')

In [ ]:
# Test
train_labels = pd.read_csv('_annot.csv')
train_labels.head()

In [ ]:
fig = plt.figure()

#add axes to the image
ax = fig.add_axes([0,0,1,1])

# read and plot the image
img_name = '1478020385196089931_jpg.rf.8fa4276fef7ddbc3af2e570ee6d0d818.jpg'
image = plt.imread(img_name)
plt.imshow(image)

# iterating over the image for different objects
for _,row in train_labels[train_labels.filename == img_name].iterrows():
    xmin = row.xmin
    xmax = row.xmax
    ymin = row.ymin
    ymax = row.ymax
    
    width = xmax - xmin
    height = ymax - ymin
    
    # assign different color to different classes of objects
    if row[3] == 'car':
        edgecolor = 'r'
        ax.annotate('1', xy=(xmax-40,ymin+20))
    elif row[3] == 'pedestrian':
        edgecolor = 'b'
        ax.annotate('2', xy=(xmax-40,ymin+20))
        
    # add bounding boxes to the image
    rect = patches.Rectangle((xmin,ymin), width, height, edgecolor = edgecolor, facecolor = 'none')
    
    ax.add_patch(rect)

In [ ]:
# Move images to test and validation and generate valid.csv and train.csv
os.chdir('C:\\Users\\andre\\Desktop\\Object detection project\\TFODCourse')
os.chdir('Tensorflow\\workspace\\export\\train')
no_img_id = 7030
for c in random.sample(glob.glob('14*'), round(no_img_id*0.1)):
    shutil.move(c, 'valid')
for c in random.sample(glob.glob('14*'), round(no_img_id*0.1)):
    shutil.move(c, 'test')

In [ ]:
# Creating a list of filenames from a certain directory
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [ ]:
# Getting the names of files from the train directory and savin them in a list
dirName = 'valid';
# Get the list of all files in directory tree at given path
listOfFiles_valid = getListOfFiles(dirName)

In [ ]:
# Saving the list of validation in a .csv file
df_valid = pd.DataFrame(listOfFiles_valid) 
# saving the dataframe 
df_valid.to_csv('valid.csv') 

In [ ]:
# Getting the names of files from the train directory and savin them in a list
dirName = 'test';
# Get the list of all files in directory tree at given path
listOfFiles_test = getListOfFiles(dirName)

In [ ]:
# Saving the list of test in a .csv file
df_test = pd.DataFrame(listOfFiles_test) 
# saving the dataframe 
df_test.to_csv('test.csv') 